# Задание

- Скачайте текст войны и мир (ссылка на [github](https://github.com/netology-code/ds3-spring-2018/tree/master/7.%20Time_series/7.4/war_and_peace) и [гугл диск](https://drive.google.com/drive/u/2/folders/1aM7cNI6Qs7AyeYpQ8lNg7vLsHzIl0JbU)). Очистите его, оставив только русские буквы и символы пробела. Постройте матрицу переходных вероятностей. Посчитайте вероятность встретить строку из 4-х символов, которая начинается п, а кончается на р; Используя матрицу переходных вероятностей сгенерируйте слово, начинающееся с буквы "Б" и имеющее длину 8-мь символов
- [Выберите ряд](https://datamarket.com/data/list/?q=cat:fpg%20provider:tsdl) и оцените его через HMM. Попробуйте "построить" (через среднее каждой компоненты) наивный прогноз ряда.

In [1]:
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
sns.set()

## Задание 1

#### Скачайте текст войны и мир (ссылка на github и гугл диск).

In [2]:
#Соединяю все файлы в один
import glob 

read_files = glob.glob("*.txt") 
with open("war_and_peace.txt", "wb") as outfile:
    for f in read_files: 
        with open(f, "rb") as infile: outfile.write(infile.read()) 

#### Очистите его, оставив только русские буквы и символы пробела.

In [3]:
with open('war_and_peace.txt', 'r', encoding = 'UTF8') as f:
    text = f.read()
    
#pattern = re.compile('[^а-я][А-Я]\s')    
pattern = re.compile('[а-яА-ЯёЁ]\s]')

text = re.sub(pattern,'', text)
text = re.sub(' ','_', text) #заменю " " на "_"

In [4]:
#проверяю текст на 'ё'
result = re.findall('ё', text)
len(result)

#На "Ё" так же проверял :)

4060

In [5]:
#Генерю алфавит (не интересно было набирать в ручную) :)
a = ord('а')
alphabet_low = ''.join([chr(i) for i in range(a,a+6)] + [chr(a+33)] + [chr(i) for i in range(a+6,a+32)])
a = ord('А')
alphabet_UP = ''.join([chr(i) for i in range(a,a+6)] + [chr(a+33)] + [chr(i) for i in range(a+6,a+32)])
alphabet = alphabet_low + '_' + alphabet_UP  
alphabet

'абвгдеёжзийклмнопрстуфхцчшщъыьэюя_АБВГДЕбЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'

#### Постройте матрицу переходных вероятностей.

In [6]:
POS = {l: i for i, l in enumerate(alphabet)}

probabilities = np.zeros((len(alphabet), len(alphabet)))
for cp, cn in zip(text[:-1], text[1:]):
    probabilities[POS[cp], POS[cn]] += 1
probabilities = (probabilities / probabilities.sum(axis=1)[:, None])

KeyError: 'A'

In [ ]:
df = pd.DataFrame(probabilities, index=(c for c in alphabet), columns=(c for c in alphabet))
df.head()

In [ ]:
plt.figure(figsize=(20,14))
sns.heatmap(df)
plt.show()

Заготовка ниже
#### Посчитайте вероятность встретить строку из 4-х символов, которая начинается п, а кончается на р;

In [ ]:
start = POS['п']
end = POS['р']
prior = 1. / len(alphabet)
for c in alphabet:
    proba = prior * probabilities[start, POS[c]] * probabilities[POS[c], end]
    if proba > 0:
        print(''.join(['п', c, 'р']), proba)

#### Используя матрицу переходных вероятностей сгенерируйте слово, начинающееся с буквы "Б" и имеющее длину 8-мь символов

##  Задание 2
### Выберите ряд и оцените его через HMM. Попробуйте "построить" (через среднее каждой компоненты) наивный прогноз ряда.

In [ ]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.tsa.api as smt

from statsmodels.tsa.stattools import adfuller

from scipy.stats import boxcox

import matplotlib.pyplot as plt
%matplotlib inline

from arch import arch_model

In [ ]:
#Исходный TS
series = pd.read_csv("Series/monthly-sales-of-company-x-jan-6.csv")['Count']

In [ ]:
import numpy as np
from hmmlearn import hmm
np.random.seed(42)

model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.6, 0.3, 0.1])
model.transmat_ = np.array([[0.7, 0.2, 0.1],
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])
model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
model.covars_ = np.tile(np.identity(2), (3, 1, 1))
X, Z = model.sample(100)

In [ ]:
f, axs = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))
_ = axs.plot(X[:, 1], X[:, 0], '.-')
_ = axs.plot(model.means_[:, 1], model.means_[:, 0], 'ro', ms=20)
_ = axs.legend()
axs.set_aspect('equal')